In [1]:
!pip install docling chromadb sentence-transformers

In [2]:
!pip install pymupdf tqdm spacy
!python -m spacy download it_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 138.8 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')


In [ ]:
!pip install transformers


In [3]:
import fitz  # PyMuPDF
from tqdm.auto import tqdm
import pandas as pd

def text_formatter(text: str) -> str:
    # Pulizia semplice
    import re
    text = text.replace("\n", " ").strip()
    text = re.sub(r"[ \t]{2,}", " ", text)
    text = re.sub(r"\.{2,}", " ", text)  # sostituisce ... con spazio
    text = re.sub(r"Pagina\s+\d+\s+di\s+\d+", "", text, flags=re.IGNORECASE)
    text = re.sub(r"Creazione VM su Cloud INSIEL","", text)
    text = re.sub(r"IO_XX_00_XX  ISTRUZIONE OPERATIVA 22/10/2024", "",text)
    text = re.sub(r"IO_XX_00_XX  ISTRUZIONE OPERATIVA 22/10/2024 ","",text)
    return text.strip()

def open_and_read_pdf(pdf_path: str):
    doc = fitz.open(pdf_path)
    pages = []
    for page_number, page in tqdm(enumerate(doc), total=len(doc), desc="📄 Lettura pagine PDF"):
        text = text_formatter(page.get_text())
        pages.append({
            "page_number": page_number + 1,
            "page_char_count": len(text),
            "page_word_count": len(text.split()),
            "page_token_estimate": len(text) // 4,
            "text": text
        })
    return pages

pdf_path = "data/insiel.pdf"  # Cambia se il tuo file è altrove
pages_and_texts = open_and_read_pdf(pdf_path)


📄 Lettura pagine PDF:   0%|          | 0/66 [00:00<?, ?it/s]

In [4]:
import spacy
nlp = spacy.load("it_core_news_sm")

# Spezza il testo di ogni pagina in frasi
for page in tqdm(pages_and_texts, desc="✂️ Split in frasi"):
    doc = nlp(page["text"])
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    page["sentence_chunks"] = []

    CHUNK_SIZE = 10  # Gruppi da 5 frasi
    for i in range(0, len(sentences), CHUNK_SIZE):
        chunk = sentences[i:i + CHUNK_SIZE]
        page["sentence_chunks"].append(chunk)


✂️ Split in frasi:   0%|          | 0/66 [00:00<?, ?it/s]

In [5]:
pages_and_texts[65]

{'page_number': 66,
 'page_char_count': 45,
 'page_word_count': 9,
 'page_token_estimate': 11,
 'text': 'FIGURA 88 - VMWARE NSX - CRITERIO GRUPPO   58',
 'sentence_chunks': [['FIGURA 88 - VMWARE NSX - CRITERIO GRUPPO   58']]}

In [6]:
df = pd.DataFrame(pages_and_texts)
df["chunk_id"] = df.index.astype(str)

# Mostra i primi
df.tail()


,page_number,page_char_count,page_word_count,page_token_estimate,text,sentence_chunks,chunk_id
61,62,1301,173,325,7 APPENDICE 2 – NAMING CONVENTION 7.1 NOMI MAC...,[[7 APPENDICE 2 – NAMING CONVENTION 7.1 NOMI M...,61
62,63,344,19,86,9.3 VMWARE ARIA • HTTPS://DOCS.VMWARE.COM/IT/V...,[[9.3 VMWARE ARIA • HTTPS://DOCS.VMWARE.COM/IT...,62
63,64,1949,352,487,12 INDICE DELLE FIGURE FIGURA 1 - VMWARE VSPHE...,[[12 INDICE DELLE FIGURE FIGURA 1 - VMWARE VSP...,63
64,65,2293,446,573,FIGURA 43 – RESOURCES – MENU CONTESTUALE 30 ...,[[FIGURA 43 – RESOURCES – MENU CONTESTUALE 3...,64
65,66,45,9,11,FIGURA 88 - VMWARE NSX - CRITERIO GRUPPO 58,[[FIGURA 88 - VMWARE NSX - CRITERIO GRUPPO 58]],65


In [7]:
df.shape

(66, 7)

In [8]:
df[df['page_token_estimate'] < 60].count()

page_number            9
page_char_count        9
page_word_count        9
page_token_estimate    9
text                   9
sentence_chunks        9
chunk_id               9
dtype: int64

In [9]:
final = df[df['page_token_estimate'] > 60]

In [10]:
final.describe().round(2)

,page_number,page_char_count,page_word_count,page_token_estimate
count,57.00,57.00,57.00,57.00
mean,34.67,821.02,129.28,204.91
std,18.73,463.38,79.10,115.84
min,3.00,245.00,19.00,61.00
25%,19.00,457.00,71.00,114.00
50%,35.00,733.00,115.00,183.00
75%,51.00,1067.00,164.00,266.00
max,65.00,2293.00,446.00,573.00


In [11]:
!pip install sentence-transformers chromadb


In [28]:
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm

embedding_model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v1")

texts = final["text"].tolist()
chunk_ids = final["chunk_id"].tolist()
metadatas = [{"page": int(p)} for p in final["page_number"]]

embeddings = embedding_model.encode(texts, show_progress_bar=True)


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
import chromadb

# nuovo client
client = chromadb.PersistentClient(path="./vectorstore")

# collection
collection = client.get_or_create_collection("insiel_chunks")

# aggiunta
collection.add(
    documents=texts,
    embeddings=embeddings.tolist(),
    metadatas=metadatas,
    ids=chunk_ids
)

In [14]:
query = input("Domanda: ")
query_embedding = model.encode([query])

results = collection.query(
    query_embeddings=query_embedding,
    n_results=3  # puoi aumentare a 5, 10, ecc.
)

Domanda:  ciao


In [15]:
for i, (doc, meta) in enumerate(zip(results["documents"][0], results["metadatas"][0])):
    print(f"\n🔹 RISULTATO {i+1} (pagina {meta['page']}):")
    print(doc[:500] + "...\n---")



🔹 RISULTATO 1 (pagina 41):
IO_XX_00_XX  ISTRUZIONE OPERATIVA 22/10/2024 o 17 macchine virtuali (vedi PAR. 5.4.1.1.4.4). o 0 profili di accesso a Livello 7 (vedi PAR. 5.4.1.1.4.3). o 2 sevizi (vedi PAR 5.4.1.1.4.1) o 6 profili contesto (vedi PAR. 5.4.1.1.4.3). 5.4.1.1.2.2 AVVISI Nel tab ‘Avvisi’ sono presenti due sotto tab: • Avvisi • Definizione avvisi Figura 60 - VMWare NSX – Home – Avvisi Figura 61 - VMWare NSX – Home – Definizione avvisi...
---

🔹 RISULTATO 2 (pagina 60):
IO_XX_00_XX  ISTRUZIONE OPERATIVA 22/10/2024 5.5.3.1.1 MICROSOFT WINDOWS SERVER 5.5.3.1.2 LINUX 5.5.3.2 AGGIUNTA A DOMINIO 5.5.3.3 ESPANSIONE DISCO DI SISTEMA 5.5.3.4 AGGIORNAMENTO SISTEMA OPERATIVO 5.5.3.4.1 VERIFICA SU CONSOLE WSUS 5.5.3.4.2 AGGIORNAMENTO E PATCHING 5.5.3.5 INSTALLAZIONE ANTIVIRUS 5.5.3.5.1 VERIFICA FUNZIONAMENTO SU CONSOLE ANTIVIRUS...
---

🔹 RISULTATO 3 (pagina 53):
IO_XX_00_XX  ISTRUZIONE OPERATIVA 22/10/2024 • Servizi Base Uscita I dettagli forniti per ogni regola sono: • Nome regola • ID re

In [35]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(torch.device("cpu"))

rag_chat = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200, device=-1)


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


In [47]:
def generate_rag_response_local(query, retrieved_chunks):
    context = "\n\n".join(retrieved_chunks)
    
    prompt = f"""[INST] Usa solo le informazioni fornite nel contesto qui sotto per rispondere alla domanda, la risposta deve finire sempre con un punto. 
Se la risposta non è presente, di' chiaramente che non è specificato nel documento.

Contesto:
{context}

Domanda: {query}
Risposta: [/INST]
"""
    result = rag_chat(prompt)[0]["generated_text"]
    return result.split("Risposta:")[-1].strip()


In [1]:
# 🧠 Inserisci la domanda
query = input("Domanda: ")

# 🔎 Ottieni l'embedding della query (usa sentence-transformers, NON il modello generativo!)
query_embedding = embedding_model.encode([query])  

# 🔍 Retrieval dei chunk più simili da Chroma
results = collection.query(
    query_embeddings=query_embedding,
    n_results=3
)

# 🧱 Estrai i chunk di contesto
retrieved_chunks = results["documents"][0]

# 🤖 Genera la risposta usando il modello open-source locale
response = generate_rag_response_local(query, retrieved_chunks)

# 🖨️ Mostra la risposta
print("🤖 Risposta:\n", response)


Domanda:  ciao


NameError: name 'embedding_model' is not defined

In [45]:
retrieved_chunks


['IO_XX_00_XX  ISTRUZIONE OPERATIVA 22/10/2024 5.3 CLOUD INSIEL 5.3.1 ACCESSO AL CLOUD INSIEL 5.3.1.1 LOGIN L’accesso al cloud INSIEL viene effettuato all’indirizzo: HTTPS://CLOUDSERVICE.INSIEL.IT/SAAS/AUTH/LOGIN NOTA BENE: • Alcune funzionalità sono raggiungibili anche al di fuori della rete ASUGI o in assenza della VPN ASUGI; tuttavia, per un corretto funzionamento è sempre preferibile accedere da un terminale connesso alla rete ASUGI, o utilizzando la VPN ASUGI. Figura 7 - Accesso al cloud INSIEL – Inserimento user name L’autenticazione avviene inserendo in un primo momento lo user name, fornito da INSIEL, e su una schermata successiva la password:',
 "IO_XX_00_XX  ISTRUZIONE OPERATIVA 22/10/2024 5.4 CLOUD FIREWALL 5.4.1.1.1 ACCESSO AL CLOUD FIREWALL Selezionando ‘Cloud Firewall’: Figura 57 - Tasto 'Cloud Firewall' Si avrà accesso ad una schermata ulteriore: Figura 58 - Accesso a Cloud Firewall Da qui cliccando sull’icona ‘Apri’: si avrà finalmente accesso all’interfaccia utente di 

In [46]:
results

{'ids': [['10', '38', '15']],
 'embeddings': None,
 'documents': [['IO_XX_00_XX  ISTRUZIONE OPERATIVA 22/10/2024 5.3 CLOUD INSIEL 5.3.1 ACCESSO AL CLOUD INSIEL 5.3.1.1 LOGIN L’accesso al cloud INSIEL viene effettuato all’indirizzo: HTTPS://CLOUDSERVICE.INSIEL.IT/SAAS/AUTH/LOGIN NOTA BENE: • Alcune funzionalità sono raggiungibili anche al di fuori della rete ASUGI o in assenza della VPN ASUGI; tuttavia, per un corretto funzionamento è sempre preferibile accedere da un terminale connesso alla rete ASUGI, o utilizzando la VPN ASUGI. Figura 7 - Accesso al cloud INSIEL – Inserimento user name L’autenticazione avviene inserendo in un primo momento lo user name, fornito da INSIEL, e su una schermata successiva la password:',
   "IO_XX_00_XX  ISTRUZIONE OPERATIVA 22/10/2024 5.4 CLOUD FIREWALL 5.4.1.1.1 ACCESSO AL CLOUD FIREWALL Selezionando ‘Cloud Firewall’: Figura 57 - Tasto 'Cloud Firewall' Si avrà accesso ad una schermata ulteriore: Figura 58 - Accesso a Cloud Firewall Da qui cliccando sull